In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyproj
import fiona
import geopandas

In [9]:
lic_comm = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/1825fde3-a668-47e3-a46d-b62b5af6560a", encoding = 'Latin-1', sep = ';')

HTTPError: HTTP Error 404: NOT FOUND

In [8]:
print(lic_comm.shape)# la base compte 68 variables au total
lic_comm=lic_comm.drop(columns=['l_qp_2019','l_qp_f_2019','l_qp_h_2019'])# variales non pertinentes pour notre étude
print(lic_comm.shape)# la base compte 68 variables au total

(939328, 54)


KeyError: "['l_qp_2019', 'l_qp_f_2019', 'l_qp_h_2019'] not found in axis"

###  Verification et Conversion des types des donnees###

la plupart des variables ont été déclarées objet cad string  au lieu de int donc il nous faut les convertir
Egalement, fed_2019 qui est un code a été enegistré comme int, donc on la convertit en string

In [6]:
print(lic_comm.dtypes) 

code_commune       object
libelle            object
region             object
fed_2019            int64
nom_fed            object
                    ...  
poph_20_29_2018    object
poph_30_44_2018    object
poph_45_59_2018    object
poph_60_74_2018    object
poph_75_2018       object
Length: 65, dtype: object


In [7]:
lic_comm['fed_2019']=lic_comm['fed_2019'].astype(str)
print(lic_comm.select_dtypes(include=object).columns)# liste des variables dont le type est string=objet
pop_object=lic_comm.select_dtypes(include=object).columns.difference(['code_commune','libelle','region','fed_2019','nom_fed'])
print(pop_object)
print(lic_comm.select_dtypes(include=int).columns)#liste des variables de type int
#for variable in type_int :
  #  lic_comm.loc[variable]=pd.to_numeric(lic_comm[variable].str.replace(',','.'),errors='coerce').astype('Int64')
#print(lic_comm.dtypes)

Index(['code_commune', 'libelle', 'region', 'fed_2019', 'nom_fed',
       'popf_0_4_2018', 'popf_5_9_2018', 'popf_10_14_2018', 'popf_15_19_2018',
       'popf_20_29_2018', 'popf_30_44_2018', 'popf_45_59_2018',
       'popf_60_74_2018', 'popf_75_2018', 'poph_0_4_2018', 'poph_5_9_2018',
       'poph_10_14_2018', 'poph_15_19_2018', 'poph_20_29_2018',
       'poph_30_44_2018', 'poph_45_59_2018', 'poph_60_74_2018',
       'poph_75_2018'],
      dtype='object')
Index(['popf_0_4_2018', 'popf_10_14_2018', 'popf_15_19_2018',
       'popf_20_29_2018', 'popf_30_44_2018', 'popf_45_59_2018',
       'popf_5_9_2018', 'popf_60_74_2018', 'popf_75_2018', 'poph_0_4_2018',
       'poph_10_14_2018', 'poph_15_19_2018', 'poph_20_29_2018',
       'poph_30_44_2018', 'poph_45_59_2018', 'poph_5_9_2018',
       'poph_60_74_2018', 'poph_75_2018'],
      dtype='object')
Index(['l_2019', 'l_0_4_2019', 'l_5_9_2019', 'l_10_14_2019', 'l_15_19_2019',
       'l_20_29_2019', 'l_30_44_2019', 'l_45_59_2019', 'l_60_74_2019',

In [8]:
print(lic_comm.dtypes)
#print(lic_comm.select_dtypes(include=object).columns)# liste des variables dont le type est string=objet

code_commune       object
libelle            object
region             object
fed_2019           object
nom_fed            object
                    ...  
poph_20_29_2018    object
poph_30_44_2018    object
poph_45_59_2018    object
poph_60_74_2018    object
poph_75_2018       object
Length: 65, dtype: object


In [9]:
lic_comm.head(5)

,code_commune,libelle,region,fed_2019,nom_fed,l_2019,l_0_4_2019,l_5_9_2019,l_10_14_2019,l_15_19_2019,...,poph_2018,poph_0_4_2018,poph_5_9_2018,poph_10_14_2018,poph_15_19_2018,poph_20_29_2018,poph_30_44_2018,poph_45_59_2018,poph_60_74_2018,poph_75_2018
0,1001,Abergement-Clémenciat,Auvergne-Rhône-Alpes,101,FF d'athlétisme,8,0,0,4,2,...,388.0,15,30,20,29,25,70,122,51,26
1,1001,Abergement-Clémenciat,Auvergne-Rhône-Alpes,103,FF de badminton,11,0,0,4,1,...,388.0,15,30,20,29,25,70,122,51,26
2,1001,Abergement-Clémenciat,Auvergne-Rhône-Alpes,105,FF de basketball,1,0,0,0,0,...,388.0,15,30,20,29,25,70,122,51,26
3,1001,Abergement-Clémenciat,Auvergne-Rhône-Alpes,108,FF de cyclisme,2,0,1,1,0,...,388.0,15,30,20,29,25,70,122,51,26
4,1001,Abergement-Clémenciat,Auvergne-Rhône-Alpes,109,FF d'équitation,10,0,0,2,2,...,388.0,15,30,20,29,25,70,122,51,26


In [10]:
lic_comm.describe(include='all')

,code_commune,libelle,region,fed_2019,nom_fed,l_2019,l_0_4_2019,l_5_9_2019,l_10_14_2019,l_15_19_2019,...,poph_2018,poph_0_4_2018,poph_5_9_2018,poph_10_14_2018,poph_15_19_2018,poph_20_29_2018,poph_30_44_2018,poph_45_59_2018,poph_60_74_2018,poph_75_2018
count,864717.0,864717,864717,864717,864717,864717.000000,864717.000000,864717.000000,864717.000000,864717.000000,...,864384.000000,864384,864384,864384,864384,864384,864384,864384,864384,864384
unique,34893.0,32478,18,99,99,NaN,NaN,NaN,NaN,NaN,...,NaN,1212,1283,1283,1235,1428,1669,1755,1555,1354
top,13055.0,Saint-Sauveur,Auvergne-Rhône-Alpes,111,FF de football,NaN,NaN,NaN,NaN,NaN,...,NaN,5,5,5,5,5,10,20,15,10
freq,95.0,341,118374,32955,32955,NaN,NaN,NaN,NaN,NaN,...,NaN,62601,53761,50992,68607,60621,23262,18331,22630,57820
mean,NaN,NaN,NaN,NaN,NaN,17.152937,0.156384,2.561943,3.927872,2.279147,...,2214.306313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,93.798889,2.728242,22.297003,28.530779,22.960352,...,8831.457310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,...,225.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,3.000000,0.000000,0.000000,0.000000,0.000000,...,530.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,9.000000,0.000000,1.000000,1.000000,1.000000,...,1453.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# valeur manquante unquement sur les données populationnelles et les variables des quartiers prooritaires

# analyse exploratoire#

**** taux de licences par regions****
impossible pour le moment car je n'arrive pas à convertir les types  

In [11]:
lic_region= lic_comm[['region','code_commune','pop_2018','l_2019','l_f_2019','l_h_2019']].drop_duplicates(['region','code_commune'])
lic_region

,region,code_commune,pop_2018,l_2019,l_f_2019,l_h_2019
0,Auvergne-Rhône-Alpes,1001,770.0,8,3,5
39,Auvergne-Rhône-Alpes,1002,271.0,1,0,1
66,Auvergne-Rhône-Alpes,1004,14217.0,86,38,48
136,Auvergne-Rhône-Alpes,1005,1718.0,2,1,1
182,Auvergne-Rhône-Alpes,1006,105.0,1,0,1
...,...,...,...,...,...,...
864612,Mayotte,97613,NaN,4,1,3
864622,Mayotte,97614,NaN,32,3,29
864641,Mayotte,97615,NaN,2,0,2
864668,Mayotte,97616,NaN,2,1,1


In [12]:
lic_region.head(20)

,region,code_commune,pop_2018,l_2019,l_f_2019,l_h_2019
0,Auvergne-Rhône-Alpes,1001,770.0,8,3,5
39,Auvergne-Rhône-Alpes,1002,271.0,1,0,1
66,Auvergne-Rhône-Alpes,1004,14217.0,86,38,48
136,Auvergne-Rhône-Alpes,1005,1718.0,2,1,1
182,Auvergne-Rhône-Alpes,1006,105.0,1,0,1
191,Auvergne-Rhône-Alpes,1007,2760.0,16,6,10
240,Auvergne-Rhône-Alpes,1008,765.0,3,2,1
280,Auvergne-Rhône-Alpes,1009,315.0,1,0,1
301,Auvergne-Rhône-Alpes,1010,1138.0,3,1,2
335,Auvergne-Rhône-Alpes,1011,377.0,1,0,1


In [13]:
lic_comm[['region', 'l_2019']].groupby('region').sum().sort_values(by = 'l_2019', ascending = False)

,l_2019
region,
Île-de-France,2361371
Auvergne-Rhône-Alpes,1991253
Nouvelle-Aquitaine,1464038
Occitanie,1399815
Grand Est,1232376
Hauts-de-France,1165458
Provence-Alpes-Côte d'Azur,1115545
Pays de la Loire,901611
Bretagne,847053


**** analyse au niveu departement******

In [14]:
lic__pop=lic_comm.drop(columns=pop_object)# supprimer les varaibles inutilisées
print(lic__pop)

       code_commune                libelle                region fed_2019  \
0              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      101   
1              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      103   
2              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      105   
3              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      108   
4              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      109   
...             ...                    ...                   ...      ...   
864712        97617               TSINGONI               Mayotte      258   
864713        97617               TSINGONI               Mayotte      261   
864714        97617               TSINGONI               Mayotte      401   
864715        97617               TSINGONI               Mayotte      403   
864716        97617               TSINGONI               Mayotte      604   

                                                  nom_fed  l_2019  l_0_4_20

importation de la base des dep puis jonture avec la base des départements

In [15]:
dep=pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25")
dep.head(10)


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes
5,1007,AMBRONAY,1500,AMBRONAY,NaN,46.005591,5.357607,7.0,NaN,Ambronay,Ambronay,1,Ain,84.0,Auvergne-Rhône-Alpes
6,1008,AMBUTRIX,1500,AMBUTRIX,NaN,45.936713,5.332809,8.0,NaN,Ambutrix,Ambutrix,1,Ain,84.0,Auvergne-Rhône-Alpes
7,1009,ANDERT ET CONDON,1300,ANDERT ET CONDON,NaN,45.787357,5.657883,9.0,NaN,Andert-et-Condon,Andert-et-Condon,1,Ain,84.0,Auvergne-Rhône-Alpes
8,1010,ANGLEFORT,1350,ANGLEFORT,NaN,45.909372,5.795160,10.0,NaN,Anglefort,Anglefort,1,Ain,84.0,Auvergne-Rhône-Alpes
9,1011,APREMONT,1100,APREMONT,NaN,46.205498,5.657815,11.0,NaN,Apremont,Apremont,1,Ain,84.0,Auvergne-Rhône-Alpes


In [16]:
dep.dtypes

code_commune_INSEE       object
nom_commune_postal       object
code_postal               int64
libelle_acheminement     object
ligne_5                  object
latitude                float64
longitude               float64
code_commune            float64
article                  object
nom_commune              object
nom_commune_complet      object
code_departement         object
nom_departement          object
code_region             float64
nom_region               object
dtype: object

In [17]:
lic_dep=pd.merge(lic__pop, dep[['code_commune_INSEE','latitude','longitude','code_departement','nom_departement']], 
                 left_on="code_commune", right_on="code_commune_INSEE")

In [18]:
lic_dep.head(5)

,code_commune,libelle,region,fed_2019,nom_fed,l_2019,l_0_4_2019,l_5_9_2019,l_10_14_2019,l_15_19_2019,...,pop_45_59_2018,pop_60_74_2018,pop_75_2018,popf_2018,poph_2018,code_commune_INSEE,latitude,longitude,code_departement,nom_departement
0,29141,Loqueffret,Bretagne,111,FF de football,2,0,0,1,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
1,29141,Loqueffret,Bretagne,115,FF de handball,3,0,0,0,2,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
2,29141,Loqueffret,Bretagne,117,FF de judo-jujitsu et disciplines associées,2,0,0,1,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
3,29141,Loqueffret,Bretagne,118,FF de lutte et disciplines associées,3,0,1,1,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
4,29141,Loqueffret,Bretagne,119,FF de natation,1,0,0,0,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère


In [24]:
lic_dep=lic_dep.drop_duplicates()
test_base=lic_dep.groupby(['code_departement','code_commune','fed_2019']).sum()
test_base

libelle  \
code_departement code_commune                                                      
29               29141         LoqueffretLoqueffretLoqueffretLoqueffretLoquef...   
                 29142         LotheyLotheyLotheyLotheyLotheyLotheyLotheyLoth...   
                 29143         MahalonMahalonMahalonMahalonMahalonMahalonMaha...   
                 29144         MartyreMartyreMartyreMartyreMartyreMartyreMart...   
                 29145         Confort-MeilarsConfort-MeilarsConfort-MeilarsC...   
...                                                                          ...   
30               30205         PougnadoressePougnadoressePougnadoressePougnad...   
                 30206         PoulxPoulxPoulxPoulxPoulxPoulxPoulxPoulxPoulxP...   
                 30207         PouzilhacPouzilhacPouzilhacPouzilhacPouzilhacP...   
                 30208         PuechredonPuechredonPuechredonPuechredonPuechr...   
                 30209         PujautPujautPujautPujautPujautPujautPujautPuja...   

                                                                          region  \
code_departement code_commune                                                      
29               29141         BretagneBretagneBretagneBretagneBretagneBretag...   
                 29142         BretagneBretagneBretagneBretagneBretagneBretag...   
                 29143         BretagneBretagneBretagneBretagneBretagneBretag...   
                 29144         BretagneBretagneBretagneBretagneBretagneBretag...   
                 29145         BretagneBretagneBretagneBretagneBretagneBretag...   
...                                                                          ...   
30               30205         OccitanieOccitanieOccitanieOccitanieOccitanieO...   
                 30206         OccitanieOccitanieOccitanieOccitanieOccitanieO...   
                 30207         OccitanieOccitanieOccitanieOccitanieOccitanieO...   
                 30208             OccitanieOccitanieOccitanieOccitanieOccitanie   
                 30209         OccitanieOccitanieOccitanieOccitanieOccitanieO...   

                                                                        fed_2019  \
code_departement code_commune                                                      
29               29141                               111115117118119124133134242   
                 29142         1011021051081091111131151231241291321331351361...   
                 29143         1051061071081091111141151171231241251281341352...   
                 29144         1011071081091111121151171181191241251261271331...   
                 29145         1011031051061071091111141151171191241261281321...   
...                                                                          ...   
30               30205         1051091111151171211231321331342062182332424024...   
                 30206         1011031051061071081091101111121131151171191211...   
                 30207         1011031051081091111131151171211231251261271281...   
                 30208                                           105111112126135   
                 30209         1011021031051061071081091101111131151171191211...   

                                                                         nom_fed  \
code_departement code_commune                                                      
29               29141         FF de footballFF de handballFF de judo-jujitsu...   
                 29142         FF d'athlétismeFF d'avironFF de basketballFF d...   
                 29143         FF de basketballFF de boxeFF de canoë-kayakFF ...   
                 29144         FF d'athlétismeFF de canoë-kayakFF de cyclisme...   
                 29145         FF d'athlétismeFF de badmintonFF de basketball...   
...                                                                          ...   
30               30205         FF de basketballFF d'équitationFF de footballF...   
                 30206         FF d'at